In [ ]:
!pip install koreanize-matplotlib #한글폰트 라이브러리 설치
!apt-get install -y fonts-nanum #나눔 고딕 폰트 설치
!pip install -q gdown

import gdown
import matplotlib.pyplot as plt
import koreanize_matplotlib  # 한글 자동 설정됨
import pandas as pd

from google.colab import drive

drive.mount('/content/drive')

pd.set_option('display.max_rows', None)
# plt.rcParams['font.family'] = 'NanumGothic' #나눔 고딕 폰트 설정

In [ ]:
# CSV 파일 불러오기

transactions_file_id = '1rzQ8Bz6hGQ7IYZPWzEkaXIJXvA6pCFkq'
output_path = 'transactions_hm.csv'
gdown.download(f'https://drive.google.com/uc?id={transactions_file_id}', output_path, quiet=False)


articles = pd.read_csv('https://drive.google.com/uc?id=1OYMa1QBf6Lt2vVABrFJi8FO4UFhieVRv')
customer = pd.read_csv('https://drive.google.com/uc?id=1uyHiU-iKwJi8ht3TqBqmmFojvSc0CY_N')
transactions = pd.read_csv('transactions_hm.csv')

In [ ]:
# 각 테이블에 대한 열과 행 개수를 표현해주세요.
articles.shape # (105542, 25)
customer.shape # (1048575, 6)
transactions.shape # (1048575, 5)


In [ ]:
# 각 테이블에 대한 컬럼타입과 통계량을 보여주세요.
articles.info()
articles.describe()
customer.info()
customer.describe()
transactions.info()
transactions.describe()

In [ ]:
# 테이블을 결합하여, 데이터분석을 위한 하나의 데이터셋으로 만들어주세요.
# 모든 테이블을 결합하지 않아도 좋습니다.
# 테이블 결합 시 inner join 으로 진행해주세요.
df_total_table = pd.merge(transactions, customer, on='customer_id', how='inner')
df_total_table = pd.merge(df_total_table, articles, on='article_id', how='inner')
df_total_table.head()

In [ ]:
# NULL 값이 존재하는 경우, 대체 or 제거 모두 가능합니다.
articles.isnull().sum() # 제품 상세 설명 null 이므로 대체/제거 하지 않음
customer.isnull().sum() # 뉴스 레터 주기 null 값이나 큰 영향을 주지 않아 대체/제거 하지 않음.
transactions.isnull().sum() # null 값 없음

In [ ]:
df_total_table["t_dat"].head()

In [ ]:
df_total_table["t_datetime"] = pd.to_datetime(df_total_table["t_dat"])
df_total_table["month"] = df_total_table["t_datetime"].dt.month

# 실제 금액처럼 보이게 price 정규화 해제 (예: 100,000 곱하기)
# df_total_table["price_real"] = df_total_table["price"] * 100000

# 월 별 판매량과 주문 수량 집계
df_total_table.groupby("month").agg(
    total_sales=("price", "sum"),
    order_count=("price", "count")
).sort_index()

# 월 별 통계량
# df_total_table.groupby("month")["price"].agg(["mean", "median", "max", "min", "count"])

In [ ]:
import matplotlib.pyplot as plt

# 월별 집계
monthly_summary = df_total_table.groupby("month").agg(
    total_sales=("price", "sum"),
    order_count=("price", "count")
).sort_index()

# 통계값 계산
mean_sales = monthly_summary['total_sales'].mean()
median_sales = monthly_summary['total_sales'].median()
mean_orders = monthly_summary['order_count'].mean()
median_orders = monthly_summary['order_count'].median()

# 시각화 시작
fig, ax1 = plt.subplots(figsize=(12, 6))

# ① 실선: 판매 매출
ax1.plot(
    monthly_summary.index,
    monthly_summary['total_sales'],
    color='tab:blue',
    marker='o',
    linestyle='-',
    label='Total Sales'
)
ax1.axhline(mean_sales, color='tab:blue', linestyle=':', label='Sales Mean')

# ✅ 매출 데이터 라벨 (왼쪽으로 살짝 이동)
for x, y in zip(monthly_summary.index, monthly_summary['total_sales']):
    ax1.text(x - 0.15, y, f'{y:,.0f}', color='tab:blue', fontsize=9, ha='right', va='bottom')

ax1.set_xlabel('Month')
ax1.set_ylabel('Total Sales (₩)', color='tab:blue')
ax1.tick_params(axis='y', labelcolor='tab:blue')
ax1.set_xticks(monthly_summary.index)

# ② 실선: 주문량
ax2 = ax1.twinx()
ax2.plot(
    monthly_summary.index,
    monthly_summary['order_count'],
    color='tab:red',
    marker='s',
    linestyle='-',
    label='Order Count'
)
ax2.axhline(mean_orders, color='tab:red', linestyle=':', label='Order Mean')

# ✅ 주문 수 데이터 라벨 (오른쪽으로 살짝 이동)
for x, y in zip(monthly_summary.index, monthly_summary['order_count']):
    ax2.text(x + 0.15, y, f'{int(y)}', color='tab:red', fontsize=9, ha='left', va='bottom')

ax2.set_ylabel('Order Count', color='tab:red')
ax2.tick_params(axis='y', labelcolor='tab:red')

# 제목
plt.title('월별 판매 매출 & 주문량 (평균선/중앙값 포함 + 좌우 분리 라벨)', fontsize=14)
fig.tight_layout()

# 범례 병합
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
fig.legend(lines1 + lines2, labels1 + labels2, loc='upper center', ncol=3)

plt.show()


In [ ]:
import matplotlib.pyplot as plt

# 색상별 월별 판매량 집계
color_month_counts = df_total_table.groupby(['month', 'perceived_colour_master_name']).size().reset_index(name='count')

# 전체적으로 가장 많이 팔린 색상 Top 10
top_colors = color_month_counts.groupby('perceived_colour_master_name')['count'].sum().nlargest(10).index

# Top 10 색상만 필터링
filtered = color_month_counts[color_month_counts['perceived_colour_master_name'].isin(top_colors)]

# Pivot: 월을 index로, 색상별 컬럼으로
pivot_df = filtered.pivot(index='month', columns='perceived_colour_master_name', values='count').fillna(0).sort_index()


# 색상 이름 매핑 (matplotlib이 인식 가능한 이름 또는 HEX)
color_map = {
    'Red': 'red',
    'Blue': 'blue',
    'Green': 'green',
    'Black': 'black',
    'White': 'gray',
    'Yellow': 'yellow',
    'Pink': 'pink',
    'Orange': 'orange',
    'Khaki green': '#7e805d',
    'Beige': '#c8ad7f',
}

# NaN 처리
pivot_df = pivot_df.fillna(0)

plt.figure(figsize=(14, 8))

# 컬럼명 반복
for color in pivot_df.columns:
    line_color = color_map.get(color, 'gray')  # 매핑 없으면 회색
    try:
        plt.plot(pivot_df.index, pivot_df[color], marker='o', label=color, color=line_color)
    except Exception as e:
        print(f"Error plotting {color}: {e}")

plt.title('Top 10 색상별 월별 판매량 추이', fontsize=16)
plt.xlabel('월', fontsize=12)
plt.ylabel('판매 건수', fontsize=12)
plt.xticks(range(1, 13))
plt.legend(title='색상', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
# 계절 컬럼 생성
df_total_table['season'] = df_total_table['month'].map({
    12: '겨울', 1: '겨울', 2: '겨울',
    3: '봄', 4: '봄', 5: '봄',
    6: '여름', 7: '여름', 8: '여름',
    9: '가을', 10: '가을', 11: '가을'
})

# 색상별 계절별 판매량 집계
color_season_counts = df_total_table.groupby(['season', 'perceived_colour_master_name']).size().reset_index(name='count')

# Top 10 색상 기준 필터링
color_season_counts = color_season_counts[color_season_counts['perceived_colour_master_name'].isin(top_colors)]

# 피벗 테이블 생성
pivot_season = color_season_counts.pivot(index='season', columns='perceived_colour_master_name', values='count').fillna(0)

# 계절 순서 지정
season_order = ['봄', '여름', '가을', '겨울']
pivot_season = pivot_season.loc[season_order]


plt.figure(figsize=(12, 6))

for color in pivot_season.columns:
    line_color = color_map.get(color, 'gray')  # color_map에서 색상 가져오기, 없으면 회색
    plt.plot(pivot_season.index, pivot_season[color], marker='o', label=color, color=line_color)

plt.title('Top 10 색상별 계절별 판매량 추이', fontsize=16)
plt.xlabel('계절', fontsize=12)
plt.ylabel('판매 건수', fontsize=12)
plt.legend(title='색상', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
# 기준컬럼을 두고 집계함수를 사용하여 비교분석을 진행해주세요. 그리고 이를 시각화 해주세요.
# 최소 1개의 인사이트를 제시해야 합니다.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# 조합별 전체 구매 건수 집계
combo_count_total = df_total_table.groupby(
    ['sales_channel_id', 'product_type_name']
).size().reset_index(name='count')

# 채널 이름 매핑
channel_map = {1: '오프라인', 2: '온라인'}

# 최대 건수 구해서 x축 통일
max_count = combo_count_total['count'].max()

# Subplots 준비
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16, 8), sharex=True)

# 그래프 그리기
for i, channel_id in enumerate([1, 2]):
    group = combo_count_total[combo_count_total['sales_channel_id'] == channel_id]
    top_combos = group.sort_values('count', ascending=False).head(10)

    sns.barplot(
        data=top_combos,
        y='product_type_name',
        x='count',
        palette='viridis',
        ax=axes[i]
    )

    axes[i].set_title(f"{channel_map[channel_id]} 채널", fontsize=14)
    axes[i].set_xlabel('Purchase Count')
    axes[i].set_ylabel('Product Type')
    axes[i].set_xlim(0, max_count * 1.1)  # 동일한 x축 범위

# 전체 타이틀과 레이아웃 조정
plt.suptitle('Top 10 Product Types by Purchase Count per Channel', fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# 1. 계절, 채널 컬럼 생성
df_total_table['month'] = pd.to_datetime(df_total_table['t_dat']).dt.month
season_map = {12: '겨울', 1: '겨울', 2: '겨울',
              3: '봄', 4: '봄', 5: '봄',
              6: '여름', 7: '여름', 8: '여름',
              9: '가을', 10: '가을', 11: '가을'}
df_total_table['season'] = df_total_table['month'].map(season_map)

channel_map = {1: '오프라인', 2: '온라인'}
df_total_table['channel'] = df_total_table['sales_channel_id'].map(channel_map)

# 2. 계절 + 채널 + 상품유형별 판매량 집계
season_channel_product_counts = df_total_table.groupby(
    ['season', 'channel', 'product_type_name']
).size().reset_index(name='count')

# 3. 계절별, 채널별 Top 10 상품 뽑기
top10_season_channel_products = season_channel_product_counts.groupby(['season', 'channel']).apply(
    lambda x: x.nlargest(10, 'count')
).reset_index(drop=True)

# 4. 전체 기준으로 정렬 순서 지정
overall_order = top10_season_channel_products.groupby('product_type_name')['count'].sum().sort_values(ascending=False).index.tolist()

# 현재 데이터에 포함된 상품만 걸러서 순서 리스트 만들기
filtered_order = [ptype for ptype in overall_order if ptype in top10_season_channel_products['product_type_name'].unique()]

# 5. 그래프 그리기
g = sns.catplot(
    data=top10_season_channel_products,
    x='count',
    y='product_type_name',
    col='season',
    hue='channel',
    kind='bar',
    col_order=['봄', '여름', '가을', '겨울'],
    height=5,
    aspect=0.8,
    dodge=True,
    order=filtered_order
)

g.set_titles("{col_name} 판매량 Top 10")
g.set_axis_labels("판매 건수", "상품 유형")

# y축 글자 작게 (상품명이 길면 유용)
for ax in g.axes.flatten():
    ax.tick_params(axis='y', labelsize=8)

plt.tight_layout()
plt.show()
